# Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ruptures as rpt

import pickle
import os
import math
import mysql.connector

from os import listdir, walk
from os.path import isfile, join

# Connect to MySQL Database

In [4]:
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='slow675',
    database='amd_project'
    )

In [5]:
print(mydb)

In [6]:
mycursor = mydb.cursor()

# Load Dataset

In [7]:
datapath = '../../DDC_Data/'
basepath = '../../'

subj_range = np.hstack((np.arange(2001,2002),np.arange(3001,3006)))
all_patients = [str(i) for i in subj_range]

In [8]:
cleaned_data_path = datapath + 'cleaned/cleaned_data_' + all_patients[0] + '_to_' + all_patients[-1] + '.csv'
predicted_data_path = datapath + 'prediction/predicted_data_' + all_patients[0] + '_to_' + all_patients[-1] + '.csv'
all_day_summary_path = datapath + 'summary/all_day_summary_' + all_patients[0] + '_to_' + all_patients[-1] + '.csv'
act_period_path = datapath + 'summary/activity_period_' + all_patients[0] + '_to_' + all_patients[-1] + '.csv'

In [9]:
%run data_cleanse.ipynb

Loading 2001's data
Loading 3001's data
Loading 3002's data
Loading 3003's data
Loading 3004's data
Loading 3005's data
Finished Loading


# Predict Labels

In [19]:
%run predict.ipynb

(0,) 1 4165 (4165, 6)
(1, 4165) 2 2762 (2762, 6)
(2,) 3 2157 (2157, 6)
(3,) 4 2426 (2426, 6)
(4,) 5 2642 (2642, 6)
(5,) 6 2426 (2426, 6)


In [20]:
y_pred_all.shape

(16578,)

In [21]:
df_all_p_sorted.shape

(16578, 7)

# Analyze Predicted Results

In [12]:
%run summarize.ipynb

In [13]:
df_all_p_sorted

,Unnamed: 0,ID,timestamp,x,y,z,HR,y_pred
0,0,2001,2019-01-16 13:58:47.956,0.246041,0.475436,0.576183,68.205284,0
1,1,2001,2019-01-16 13:58:48.117,0.246431,0.473881,0.576429,68.205284,0
2,2,2001,2019-01-16 13:58:48.278,0.246236,0.474464,0.573722,68.205284,0
3,3,2001,2019-01-16 13:58:48.439,0.246041,0.473881,0.574460,67.365480,0
4,4,2001,2019-01-16 13:58:48.6,0.246821,0.472521,0.575444,67.365480,0
5,5,2001,2019-01-16 13:58:48.76,0.247016,0.474659,0.572246,67.365480,0
6,6,2001,2019-01-16 13:58:48.922,0.247211,0.477574,0.573722,67.365480,0
7,7,2001,2019-01-16 13:58:49.08,0.245262,0.469994,0.575198,67.365480,0
8,8,2001,2019-01-16 13:58:49.243,0.246041,0.475047,0.571754,67.365480,0
9,9,2001,2019-01-16 13:58:49.402,0.246236,0.471937,0.570277,67.365480,0


In [41]:
df_summary_all

,ID,date,from,to,from actual,to actual,sit,sleep,stand,walk,total,sit count,sleep count,stand count,walk count,inactive count,active count,total count,transition count,duration per action
0,2001,2019-01-16,13:0:0,14:0:0,13:58:47.956,14:0:0.0,0:0:39,0:0:31,0:0:00,0:0:00,0:1:10,2,2,1,0,4,1,5,2,0:12:00
1,2001,2019-01-16,14:0:0,15:0:0,14:0:0.0,14:09:58.232,0:7:05,0:2:48,0:0:00,0:0:02,0:9:55,7,7,0,1,14,1,15,2,0:4:00
2,3001,2019-03-05,10:0:0,11:0:0,10:03:24.717,10:10:48.041,0:1:03,0:6:18,0:0:00,0:0:00,0:7:21,5,6,0,0,11,0,11,0,0:5:27
3,3002,2019-03-05,12:0:0,13:0:0,12:23:37.598,12:29:23.812,0:1:14,0:4:31,0:0:00,0:0:00,0:5:45,2,3,0,0,5,0,5,0,0:12:00
4,3003,2019-03-05,12:0:0,13:0:0,12:33:59.899,12:40:29.414,0:0:00,0:6:29,0:0:00,0:0:00,0:6:29,0,1,0,0,1,0,1,0,1:60:00
5,3004,2019-03-05,12:0:0,13:0:0,12:47:04.763,12:54:08.963,0:1:09,0:5:53,0:0:00,0:0:00,0:7:02,2,3,0,0,5,0,5,0,0:12:00
6,3005,2019-03-05,12:0:0,13:0:0,12:59:27.632,13:0:0.0,0:0:00,0:0:32,0:0:00,0:0:00,0:0:32,0,1,0,0,1,0,1,0,1:60:00
7,3005,2019-03-05,13:0:0,14:0:0,13:0:0.0,13:05:57.101,0:0:00,0:5:57,0:0:00,0:0:00,0:5:57,0,1,0,0,1,0,1,0,1:60:00


In [42]:
df_act_period

,ID,date,from,to,y_pred
0,2001,2019-01-16,13:58:47.956,13:59:26.505,0
1,2001,2019-01-16,13:59:26.666,13:59:30.36,1
2,2001,2019-01-16,13:59:30.522,13:59:31.001,2
3,2001,2019-01-16,13:59:31.164,13:59:59.116,1
4,2001,2019-01-16,13:59:59.275,14:0:0.0,0
5,2001,2019-01-16,14:0:0.0,14:01:03.039,0
6,2001,2019-01-16,14:01:03.196,14:01:13.155,1
7,2001,2019-01-16,14:01:13.316,14:01:17.653,0
8,2001,2019-01-16,14:01:17.813,14:02:46.004,1
9,2001,2019-01-16,14:02:46.165,14:02:48.574,3


# All Day Data

In [22]:
cleaned_data_path = datapath + 'cleaned/cleaned_data_9999.csv'
predicted_data_path = datapath + 'prediction/predicted_data_9999.csv'
all_day_summary_path = datapath + 'summary/all_day_summary_9999.csv'
act_period_path = datapath + 'summary/activity_period_9999.csv'

In [24]:
%run copy_data.ipynb

Loading 2001's data
Loading 3001's data
Loading 3002's data
Loading 3003's data
Loading 3004's data
Loading 3005's data
Finished Loading


In [ ]:
%run predict.ipynb


 13%|█▎        | 3/23 [01:58<13:27, 40.37s/it]

In [18]:
all_patients = ['9999']

In [19]:
%run summarize.ipynb

2019-03-28 0:0:0.0 24:0:0.0
[array([[ 92,  50, 312, 280]]), array([[ 94,  54, 276, 234]]), array([[ 95,  55, 267, 235]]), array([[ 90,  51, 324, 286]]), array([[ 96,  54, 258, 222]]), array([[ 91,  55, 286, 252]]), array([[ 93,  52, 309, 269]]), array([[ 95,  52, 275, 241]]), array([[ 89,  52, 297, 264]]), array([[ 96,  52, 297, 253]]), array([[ 93,  53, 258, 225]]), array([[ 91,  51, 316, 289]]), array([[ 98,  59, 281, 233]]), array([[ 90,  52, 264, 231]]), array([[ 93,  50, 312, 280]]), array([[ 94,  54, 276, 234]]), array([[ 94,  55, 266, 235]]), array([[ 91,  51, 325, 286]]), array([[ 97,  54, 258, 222]]), array([[ 91,  55, 286, 252]]), array([[ 93,  52, 310, 269]]), array([[ 95,  52, 276, 241]]), array([[ 89,  52, 298, 265]]), array([[ 97,  52, 298, 253]])]


In [48]:
df_summary_all.shape

(24, 20)

# Insert to Database

In [37]:
sql = "INSERT INTO Patient (ID, Dateandtime, X, Y, Z, HR, Label) \
    VALUES (%s, %s, %s, %s, %s, %s, %s)"

for row in zip(df_all_p_sorted['ID'], 
               df_all_p_sorted['timestamp'], 
               df_all_p_sorted['x'], 
               df_all_p_sorted['y'], 
               df_all_p_sorted['z'], 
               df_all_p_sorted['HR'], 
               df_all_p_sorted['y_pred']):
    
    mycursor.execute(sql, row)

mydb.commit()

In [52]:
sql = "INSERT INTO AllDaySummary (ID, Date, TimeFrom, TimeUntil, ActualFrom, ActualUntil,\
    DurationSit, DurationSleep, DurationStand, DurationWalk, TotalDuration,\
    CountSit, CountSleep, CountStand, CountWalk,\
    CountActive, CountInactive,\
    CountTotalActiveness, CountTransition, DurationPerTransition)\
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,\
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

for row in zip(df_summary_all['ID'], 
               df_summary_all['date'], 
               df_summary_all['from'],
               df_summary_all['to'],
               df_summary_all['from actual'],
               df_summary_all['to actual'],
               df_summary_all['sit'], 
               df_summary_all['sleep'], 
               df_summary_all['stand'],
               df_summary_all['walk'],
               df_summary_all['total'],
               df_summary_all['sit count'], 
               df_summary_all['sleep count'], 
               df_summary_all['stand count'], 
               df_summary_all['walk count'], 
               df_summary_all['inactive count'], 
               df_summary_all['active count'], 
               df_summary_all['total count'], 
               df_summary_all['transition count'], 
               df_summary_all['duration per action']):
    
    mycursor.execute(sql, row)
    
mydb.commit()

In [53]:
sql = "INSERT INTO ActivityPeriod (ID, Date, TimeFrom, TimeUntil, Label)\
    VALUES (%s, %s, %s, %s, %s)"

for row in zip(df_act_period['ID'], 
               df_act_period['date'], 
               df_act_period['from'],
               df_act_period['to'],
               df_act_period['y_pred']):
    
    mycursor.execute(sql, row)

mydb.commit()

# Show Results

In [54]:
mycursor.execute("SELECT * FROM Patient")

myresult = mycursor.fetchall()

print(len(myresult))

for x in range(50):
    print(myresult[x])
    

540000
(9999, datetime.datetime(2019, 3, 28, 0, 0), 0.246, 0.4754, 0.5762, 68.2053, 2)
(9999, datetime.datetime(2019, 3, 28, 0, 0, 0, 160000), 0.2464, 0.4739, 0.5764, 68.2053, 2)
(9999, datetime.datetime(2019, 3, 28, 0, 0, 0, 320000), 0.2462, 0.4745, 0.5737, 68.2053, 2)
(9999, datetime.datetime(2019, 3, 28, 0, 0, 0, 480000), 0.246, 0.4739, 0.5745, 67.3655, 2)
(9999, datetime.datetime(2019, 3, 28, 0, 0, 0, 640000), 0.2468, 0.4725, 0.5754, 67.3655, 2)
(9999, datetime.datetime(2019, 3, 28, 0, 0, 0, 800000), 0.247, 0.4747, 0.5722, 67.3655, 2)
(9999, datetime.datetime(2019, 3, 28, 0, 0, 0, 960000), 0.2472, 0.4776, 0.5737, 67.3655, 2)
(9999, datetime.datetime(2019, 3, 28, 0, 0, 1, 120000), 0.2453, 0.47, 0.5752, 67.3655, 2)
(9999, datetime.datetime(2019, 3, 28, 0, 0, 1, 280000), 0.246, 0.475, 0.5718, 67.3655, 2)
(9999, datetime.datetime(2019, 3, 28, 0, 0, 1, 440000), 0.2462, 0.4719, 0.5703, 67.3655, 2)
(9999, datetime.datetime(2019, 3, 28, 0, 0, 1, 600000), 0.246, 0.4735, 0.5715, 66.9456, 2)


In [55]:
mycursor.execute("SELECT * FROM ActivityPeriod")

myresult = mycursor.fetchall()

print(len(myresult))

for x in range(len(myresult)):
    print(myresult[x])

16482
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0), datetime.timedelta(0, 35), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 35), datetime.timedelta(0, 70), 0)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 70), datetime.timedelta(0, 108), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 108), datetime.timedelta(0, 110), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 110), datetime.timedelta(0, 115), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 115), datetime.timedelta(0, 116), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 116), datetime.timedelta(0, 116), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 116), datetime.timedelta(0, 116), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 116), datetime.timedelta(0, 119), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 119), datetime.timedelta(0, 119), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 119), 

(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 614), datetime.timedelta(0, 614), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 614), datetime.timedelta(0, 615), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 615), datetime.timedelta(0, 615), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 616), datetime.timedelta(0, 616), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 616), datetime.timedelta(0, 616), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 616), datetime.timedelta(0, 617), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 617), datetime.timedelta(0, 617), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 617), datetime.timedelta(0, 618), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 618), datetime.timedelta(0, 618), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 618), datetime.timedelta(0, 625), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 625

(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 1527), datetime.timedelta(0, 1529), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 1529), datetime.timedelta(0, 1557), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 1557), datetime.timedelta(0, 1557), 1)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 1557), datetime.timedelta(0, 1561), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 1561), datetime.timedelta(0, 1567), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 1568), datetime.timedelta(0, 1572), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 1572), datetime.timedelta(0, 1577), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 1577), datetime.timedelta(0, 1584), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 1584), datetime.timedelta(0, 1586), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 1586), datetime.timedelta(0, 1586), 3)
(9999, datetime.date(2019, 3, 28), datet

(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 2772), datetime.timedelta(0, 2772), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 2772), datetime.timedelta(0, 2779), 0)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 2779), datetime.timedelta(0, 2786), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 2786), datetime.timedelta(0, 2786), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 2786), datetime.timedelta(0, 2788), 0)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 2788), datetime.timedelta(0, 2791), 1)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 2792), datetime.timedelta(0, 2793), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 2794), datetime.timedelta(0, 2797), 0)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 2798), datetime.timedelta(0, 2798), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 2798), datetime.timedelta(0, 2806), 0)
(9999, datetime.date(2019, 3, 28), datet

(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 3269), datetime.timedelta(0, 3270), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 3270), datetime.timedelta(0, 3270), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 3270), datetime.timedelta(0, 3270), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 3271), datetime.timedelta(0, 3277), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 3277), datetime.timedelta(0, 3277), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 3278), datetime.timedelta(0, 3279), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 3279), datetime.timedelta(0, 3279), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 3279), datetime.timedelta(0, 3280), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 3280), datetime.timedelta(0, 3285), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 3286), datetime.timedelta(0, 3287), 0)
(9999, datetime.date(2019, 3, 28), datet

...

(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82372), datetime.timedelta(0, 82372), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82373), datetime.timedelta(0, 82380), 0)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82380), datetime.timedelta(0, 82381), 1)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82381), datetime.timedelta(0, 82381), 0)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82381), datetime.timedelta(0, 82382), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82382), datetime.timedelta(0, 82384), 0)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82385), datetime.timedelta(0, 82386), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82386), datetime.timedelta(0, 82391), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82392), datetime.timedelta(0, 82392), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82392), datetime.timedelta(0, 82422), 1)
(9999, datetime.date

(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82852), datetime.timedelta(0, 82853), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82853), datetime.timedelta(0, 82853), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82854), datetime.timedelta(0, 82854), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82855), datetime.timedelta(0, 82860), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82860), datetime.timedelta(0, 82862), 0)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82862), datetime.timedelta(0, 82868), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82868), datetime.timedelta(0, 82868), 0)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82868), datetime.timedelta(0, 82869), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82869), datetime.timedelta(0, 82870), 0)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 82871), datetime.timedelta(0, 82871), 3)
(9999, datetime.date

(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 83799), datetime.timedelta(0, 83804), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 83804), datetime.timedelta(0, 83811), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 83811), datetime.timedelta(0, 83813), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 83813), datetime.timedelta(0, 83813), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 83813), datetime.timedelta(0, 83813), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 83814), datetime.timedelta(0, 83814), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 83814), datetime.timedelta(0, 83814), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 83815), datetime.timedelta(0, 83816), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 83816), datetime.timedelta(0, 83822), 1)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 83823), datetime.timedelta(0, 83823), 0)
(9999, datetime.date

(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85006), datetime.timedelta(0, 85012), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85013), datetime.timedelta(0, 85013), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85013), datetime.timedelta(0, 85015), 0)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85015), datetime.timedelta(0, 85018), 1)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85018), datetime.timedelta(0, 85020), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85020), datetime.timedelta(0, 85024), 0)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85024), datetime.timedelta(0, 85025), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85025), datetime.timedelta(0, 85033), 0)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85033), datetime.timedelta(0, 85034), 1)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85034), datetime.timedelta(0, 85034), 0)
(9999, datetime.date

(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85495), datetime.timedelta(0, 85496), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85496), datetime.timedelta(0, 85496), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85496), datetime.timedelta(0, 85496), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85497), datetime.timedelta(0, 85497), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85497), datetime.timedelta(0, 85497), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85497), datetime.timedelta(0, 85504), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85504), datetime.timedelta(0, 85504), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85504), datetime.timedelta(0, 85506), 3)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85506), datetime.timedelta(0, 85506), 2)
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 85506), datetime.timedelta(0, 85507), 3)
(9999, datetime.date

In [56]:
mycursor.execute("SELECT * FROM AllDaySummary")

myresult = mycursor.fetchall()

print(len(myresult))

for x in range(len(myresult)):
    print(myresult[x])

24
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0), datetime.timedelta(0, 3600), datetime.timedelta(0), datetime.timedelta(0, 3600), datetime.timedelta(0, 798), datetime.timedelta(0, 583), datetime.timedelta(0, 1435), datetime.timedelta(0, 665), datetime.timedelta(0, 3481), 92, 50, 312, 280, 142, 592, 734, 190, datetime.timedelta(0, 4))
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 3600), datetime.timedelta(0, 7200), datetime.timedelta(0, 3600), datetime.timedelta(0, 7200), datetime.timedelta(0, 781), datetime.timedelta(0, 592), datetime.timedelta(0, 1459), datetime.timedelta(0, 662), datetime.timedelta(0, 3494), 94, 54, 276, 234, 148, 510, 658, 197, datetime.timedelta(0, 5))
(9999, datetime.date(2019, 3, 28), datetime.timedelta(0, 7200), datetime.timedelta(0, 10800), datetime.timedelta(0, 7200), datetime.timedelta(0, 10800), datetime.timedelta(0, 844), datetime.timedelta(0, 634), datetime.timedelta(0, 1405), datetime.timedelta(0, 611), datetime.timedelta(0, 3494), 

In [42]:
# mycursor.execute("DELETE FROM AllDaySummary")
# mycursor.execute("DELETE FROM ActivityPeriod")
# mycursor.execute("DELETE FROM Patient")

In [50]:
# mycursor.execute("DROP TABLE AllDaySummary")
# mycursor.execute("DROP TABLE ActivityPeriod")
# mycursor.execute("DROP TABLE Patient")

# mycursor.execute("DROP DATABASE amd_project")